In [1]:
! pip3 install datasets transformers[sentencepiece]==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!pip3 install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
! pip install sacrebleu

#### All import


In [3]:
from collections import Counter, defaultdict
import torch
import torch.functional as F
import torch.nn as nn
import numpy as np
from torchtext.vocab import vocab as Vocab
import os
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import WordPunctTokenizer #try use other tokenizer
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
#from my_network import Seq2Seq, Encoder, Decoder
from IPython.display import clear_output
#from torch.utils.tensorboard import SummaryWriter
import time
#from torchmetrics import BLEUScore, Accuracy
import transformers

In [4]:
import datasets
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from random import randrange

In [5]:
from transformers.utils import send_example_telemetry
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

send_example_telemetry("translation_notebook", framework="pytorch")
model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"

In [19]:
from huggingface_hub import notebook_login
notebook_login()

#### Download data

In [21]:
path_to_data = "/content/sample_data/data.txt"
data = list(open(path_to_data, encoding="utf-8"))
data = [line.split(sep="\t") for line in data]
data = [[line[0], line[1].replace("\n", "")] for line in data]
len(data)

50000

#### Make tokenizer

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if "mbart" in model_checkpoint:
    print("okk")
    tokenizer.src_lang = "en-XX"
    print("ok2k")
    tokenizer.tgt_lang = "ru-RU"

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [23]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [24]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ru"

def preprocess_function(examples):
    inputs = []
    targets = []
    for example in examples["translation"]:
        if example[source_lang] is not None and example[target_lang] is not None and \
        len(example[source_lang].strip()) > 3 and len(example[target_lang].strip()) > 3:
            inputs.append(prefix + example[source_lang].strip())
            targets.append(example[target_lang].strip())
        else:
            "There is an issue with this segment:"
            print("Source:", example[source_lang])
            print("Target:", example[target_lang])
            random_num = randrange(10000)
            print("Replaced with", random_num)
            inputs.append(prefix + str(random_num))
            targets.append(str(random_num))
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
    return model_inputs
    

#### Load metric

In [36]:
from datasets import load_dataset, load_metric

metric = load_metric("sacrebleu")

#### Make require dataset

In [25]:
dataset_dict = {"en": [], "ru": []}
for elem in data:
    dataset_dict["en"].append(elem[0])
    dataset_dict["ru"].append(elem[1])

df = pd.DataFrame(dataset_dict)
df.to_csv(path_or_buf="dataset_en_ru.csv", index=False)

In [26]:
dataset = load_dataset("csv", data_files="/content/dataset_en_ru.csv")
dataset = dataset["train"].map(lambda ex, i: {"id": i, "translation": dict(ex)}, remove_columns=["en", "ru"], features=datasets.Features({"id": datasets.Value("string"), "translation": datasets
.Translation(languages=["en", "ru"])}), with_indices=True,)
dataset = dataset.train_test_split(test_size=0.2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [37]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

#### Other functions

In [28]:
import numpy as np
from datasets import load_metric

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

#### Make model

In [29]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [38]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-en-to-ru",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [39]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [40]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/opus-mt-en-ru-finetuned-en-to-ru is already a clone of https://huggingface.co/niktiuk/opus-mt-en-ru-finetuned-en-to-ru. Make sure you pull the latest changes with `repo.git_pull()`.


In [41]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.788600,0.724626,35.535400,27.680200


TrainOutput(global_step=2500, training_loss=0.7502045776367188, metrics={'train_runtime': 1127.695, 'train_samples_per_second': 35.471, 'train_steps_per_second': 2.217, 'total_flos': 472589817348096.0, 'train_loss': 0.7502045776367188, 'epoch': 1.0})